# 栈 队列 链表 哈希表

# 并查集

并查集的精髓即三个操作： 初始化， 查找， 合并

In [6]:
# 初始化
import numpy as np
def Initialization(n):
    father = np.linspace(1, n, n)
    return father

In [8]:
# 查找 -- 无路径压缩
def find(x):
    while father[x] != x:
        x = father[x]
    return x

# 查找 -- 路径压缩
def find(x):
    if father[x] != x: # x不是自身的父亲，即x不是该集合的代表
        father[x] = find(father[x]) # 查找x的祖先直到找到代表,于是顺手路径压缩
    return father[x]

# 合并
def connect(x, y):
    root_x = find(x)
    root_y = find(y)
    if root_x != root_y:
        father[root_x] = root_y

由于需要我们支持的只有集合的合并、查询操作，当我们需要将两个集合合二为一时，无论将哪一个集合连接到另一个集合的下面，都能得到正确的结果。但不同的连接方法存在时间复杂度的差异。具体来说，如果我们将一棵点数与深度都较小的集合树连接到一棵更大的集合树下，显然相比于另一种连接方案，其期望复杂度更优（也会带来更优的最坏复杂度）。

当然，我们不总能遇到恰好如上所述的集合————点数与深度都更小。鉴于点数与深度这两个特征都很容易维护，我们常常从中择一，作为估价函数。而无论选择哪一个，时间复杂度都为 \phi(m\apha(m,n)). Tarjan 的论文中，证明了不使用启发式合并、只使用路径压缩的最坏时间复杂度是\phi(mlog n)  。在姚期智的论文中，证明了不使用启发式合并、只使用路径压缩，在平均情况下，时间复杂度依然是\phi(m\apha(m,n))

citation https://oi-wiki.org/ds/dsu/

### 例题
Connecting Graph

Given n nodes in a graph labeled from 1 to n. There is no edges in the graph at beginning.

You need to support the following method:

connect(a, b), add an edge to connect node a and node b`.

query(a, b), check if two nodes are connected.

In [16]:
class connecting_graph():
    
    def __init__(self, n):
        self.father = list(range(n)) 
        
    def find(self, x):
        if self.father[x] == x:
            return x
        self.father[x] = self.find(self.father[x])
        return self.father[x]
    
    def connecting(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            self.father[root_x] = root_y
    
    def query(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        return root_x == root_y
    
graph = connecting_graph(3)
graph.connecting(1, 2)
print(graph.query(0, 1))
print(graph.query(2, 1))

False
True


Connecting Graph II

Given n nodes in a graph labeled from 1 to n. There is no edges in the graph at beginning.

You need to support the following method:

connect(a, b), an edge to connect node a and node b
query(a), Returns the number of connected component nodes which include node a.

In [20]:
class connecting_graph():
    
    def __init__(self, n):
        self.father = [i for i in range(n+1)] # labeled from 1 to n
        self.size = [1]*(n + 1)
        
    def find(self, x):
        if self.father[x] == x:
            return x
        self.father[x] = self.find(self.father[x])
        return self.father[x]
    
    def connecting(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            self.father[root_x] = root_y
            self.size[root_y] += self.size[root_x]
            
    def query(self, x):
        root_x = self.find(x)
        return self.size[root_x]
    
graph = connecting_graph(5)
graph.connecting(1, 2)
graph.connecting(1, 3)
graph.connecting(4, 5)
print(graph.query(1))
print(graph.query(5))

3
2


Number of Islands

Given a boolean 2D matrix, 0 is represented as the sea, 1 is represented as the island. If two 1 is adjacent, we consider them in the same island. We only consider up/down/left/right adjacent.

Find the number of islands.

这道题, 用bfs或者dfs的时候, k个操作, 时间复杂度是O(kmn), 其中m是行数, n是列数.
但是其实每次操作, 我们并不需要遍历整张图, 只需要在每个点附近的局部做处理就行, 这种bfs/dfs下的局部处理, 可以想到并查集. 需要做的的事是: 把其他的操作,转化成查找和合并这两件事情.

值得注意的是, 二位矩阵转一维有个公式:
二位矩阵中的(x, y)某点如果想变为一维矩阵中的某个位置, id转化公式如下, 其中m为二维矩阵的列数:

(x, y) ---> ID :     ID = x * m + y

ID ----> (x, y):     x = ID / m

                     y = ID % m

In [25]:
class connecting_graph():
    def __init__(self, n):
        self.father = list(range(n))
        
    def find(self, x):
        if self.father[x] == x:
            return x
        self.father[x] = self.find(self.father[x])
        return self.father[x]
    
    def connecting(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            self.father[root_x] = root_y
            self.count -= 1
    
    def set_count(self, count):
        self.count = count
        
    def query(self):
        return self.count
    

def num_of_islands(grid):
    if not grid:
        return 0
    m, n  = len(grid), len(grid[0])
    if m == 0 or n == 0:
        return 0 
    graph = connecting_graph(m * n)
    graph.set_count(sum([sum(grid[i]) for i in range(m)]))
    for i in range(m):
        for j in range(n):
            if grid[i][j]:
                if i > 0 and grid[i-1][j]:
                    graph.connecting(i * m + j, (i - 1) * m + j)
                if j < m - 1 and grid[i][j + 1]:
                    graph.connecting(i * m + j, i * m + j + 1)
                    
    return graph.query()

grid = [[1, 0, 0, 1], [0, 1, 1, 0], [0, 1, 1, 0], [1, 0, 0, 1]]
print(num_of_islands(grid))

# 合并岛屿减count 时间复杂度为: O(mn + 4k), 其中k为操作的次数, 小于bfs/dfs的O(mn*k)

5


# Trie Tree 字典树

trie，又称前缀树或字典树，是一种有序树，用于保存关联数组，其中的键通常是字符串。与二叉查找树不同，键不是直接保存在节点中，而是由节点在树中的位置决定。一个节点的所有子孙都有相同的前缀，也就是这个节点对应的字符串，而根节点对应空字符串。一般情况下，不是所有的节点都有对应的值，只有叶子节点和部分内部节点所对应的键才有相关的值

trie树实际上是一个确定有限状态自动机(DFA)，通常用转移矩阵表示。行表示状态，列表示输入字符，（行，列）位置表示转移状态。这种方式的查询效率很高，但由于稀疏的现象严重，空间利用效率很低。也可以采用压缩的存储方式即链表来表示状态转移，但由于要线性查询，会造成效率低下。

In [26]:
from typing import Tuple


class TrieNode(object):
    """
    Our trie node implementation. Very basic. but does the job
    """
    
    def __init__(self, char: str):
        self.char = char
        self.children = []
        # Is it the last character of the word.`
        self.word_finished = False
        # How many times this character appeared in the addition process
        self.counter = 1
    

def add(root, word: str):
    """
    Adding a word in the trie structure
    """
    node = root
    for char in word:
        found_in_child = False
        # Search for the character in the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found it, increase the counter by 1 to keep track that another
                # word has it as well
                child.counter += 1
                # And point the node to the child that contains this char
                node = child
                found_in_child = True
                break
        # We did not find it so add a new chlid
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            # And then point node to the new child
            node = new_node
    # Everything finished. Mark it as the end of a word.
    node.word_finished = True


def find_prefix(root, prefix: str) -> Tuple[bool, int]:
    """
    Check and return 
      1. If the prefix exsists in any of the words we added so far
      2. If yes then how may words actually have the prefix
    """
    node = root
    # If the root node has no children, then return False.
    # Because it means we are trying to search in an empty trie
    if not root.children:
        return False, 0
    for char in prefix:
        char_not_found = True
        # Search through all the children of the present `node`
        for child in node.children:
            if child.char == char:
                # We found the char existing in the child.
                char_not_found = False
                # Assign node as the child containing the char and break
                node = child
                break
        # Return False anyway when we did not find a char.
        if char_not_found:
            return False, 0
    # Well, we are here means we have found the prefix. Return true to indicate that
    # And also the counter of the last node. This indicates how many words have this
    # prefix
    return True, node.counter

if __name__ == "__main__":
    root = TrieNode('*')
    add(root, "hackathon")
    add(root, 'hack')

    print(find_prefix(root, 'hac'))
    print(find_prefix(root, 'hack'))
    print(find_prefix(root, 'hackathon'))
    print(find_prefix(root, 'ha'))
    print(find_prefix(root, 'hammer'))

(True, 2)
(True, 2)
(True, 1)
(True, 2)
(False, 0)


# 块状数据结构
## 分块思想
通常的分块算法的复杂度带根号，或者其他奇怪的复杂度，而不是log.分块是一种很灵活的思想，几乎什么都能分块，并且不难实现。你想写出什么数据结构就有什么，缺点是渐进意义的复杂度不够好。当然，在n=10^5 时，由于常数小，跟线段树可能差不多。这不是建议你们用分块的意思，只是说它可以作为一个备用方案，首选肯定是线段树等高级的数据结构。
### 区间和

# 单调栈
## 操作：
##### 插入

将一个元素插入单调栈时，为了维护栈的单调性，需要在保证将该元素插入到栈顶后整个栈满足单调性的前提下弹出最少的元素。伪代码如下：

In [ ]:
def monotone_stack_insert(stack, x):
    while stack and stack[-1] < x:
        stack.pop()
    stack.add(x)

### 单调栈伪代码模板

In [ ]:
# 伪代码
stack<int> st;
#此处一般需要给数组最后添加结束标志符，具体下面例题会有详细讲解
for (遍历这个数组)
{
    if (栈空 || 栈顶元素大于等于当前比较元素)
    {
        入栈;
    }
    else
    {
        while (栈不为空 && 栈顶元素小于当前元素)
        {
            栈顶元素出栈;
            更新结果;
        }
        当前数据入栈;
    }
}

### 例题

##### 例一
有一群牛站成一排，每头牛都是面朝右的，每头牛可以看到他右边身高比他小的牛。给出每头牛的身高，要求每头牛能看到的牛的总数。

In [9]:
def fieldsum(stack): # online! O(n)
    if len(stack) == 1:
        return 0
    result, temp = 0, []
    stack += [float('inf')]
    for i in range(len(stack)):
        while temp and stack[temp[-1]] <= stack[i]:
            result += i - temp.pop() - 1
        temp.append(i) # not temp or stack[temp[-1]] > stack[i]
    return result

print(fieldsum([4, 3, 7, 1]))

2


##### 例二
给定n个非负整数，用于表示柱状图柱高，柱宽1，求柱状图中能勾勒出来巨星最大面积

In [14]:
def largest_rectangle(heights):
    if len(heights) == 0:
        return heights[-1]
    result, idx = 0, []
    heights += [0]
    for i in range(len(heights)):
        while idx and heights[idx[-1]] >= heights[i]:
            height = heights[idx.pop()]
            width = i if not idx else i - 1 - idx[-1]
            result = max(result, height*width)
        idx.append(i)
    return result

print(largest_rectangle([2,1,5,6,2,3])) 
print(largest_rectangle([5,6,6,6,5,6]))

10
30


##### 例三
给出一组数字，求一区间，使得区间元素和乘以区间最小值最大，结果要求给出这个最大值和

单调栈也可以用于离线解决 RMQ 问题。

我们可以把所有询问按右端点排序，然后每次在序列上从左往右扫描到当前询问的右端点处，并把扫描到的元素插入到单调栈中。这样，每次回答询问时，单调栈中存储的值都是位置  的、可能成为答案的决策点，并且这些元素满足单调性质。此时，单调栈上第一个位置  的元素就是当前询问的答案，这个过程可以用二分查找实现。使用单调栈解决 RMQ 问题的时间复杂度为O(alogq + qlogn)，空间复杂度为O(n)

# 单调队列

## 概念

单调队列的重点分为 "单调" 和 "队列"。"单调" 指的是元素递增(或递减),"队列" 指的是元素只能从队头和队尾进行操作.单调队列中的"队列"与正常的队列有一定的区别，此处的队列跟普通队列的一大不同就在于可以从队尾进行操作

### 例题

##### 例一
给出一个长度为n的数组，编程输出每k个连续的数中的最大值和最小值。最暴力的想法很简单，时间复杂度O(kn).有了上面 "单调队列" 的概念，很容易想到用单调队列进行优化。

要求的是每连续的k个数中的最大（最小）值，很明显，当一个数进入所要 "寻找" 最大值的范围中时，若这个数比其前面（先进队）的数要大，显然，前面的数会比这个数先出队且不再可能是最大值。也就是说——当满足以上条件时，可将前面的数 "弹出"，再将该数真正 push 进队尾。
这就相当于维护了一个递减的队列，符合单调队列的定义，减少了重复的比较次数，不仅如此，由于维护出的队伍是查询范围内的且是递减的，队头必定是该查询区域内的最大值，因此输出时只需输出队头即可。显而易见的是，在这样的算法中，每个数只要进队与出队各一次，因此时间复杂度被降到了O(m)。而由于查询区间长度是固定的，超出查询空间的值再大也不能输出，因此还需要 site 数组记录第i个队中的数在原数组中的位置，以弹出越界的队头。

In [50]:
from collections import deque
def subarray_maximum(nums, k): # online time complexity:O(n)
    len_nums = len(nums)
    if len_nums < k:
        return []
    result = []    
    temp = deque()
    for i, num in enumerate(nums):
        while temp and temp[-1][1]<num:
            temp.pop()          
        temp.append((i,num))
        if i>=k-1:
            if temp[0][0] <= i - k:
                temp.popleft() 
            result.append(temp[0][1])
    return result

subarray_maximum([0,3,-1,-3,5,3,6,7], 3)


[3, 3, 5, 5, 6, 7]

# 树状数组
树状数组和下面的线段树可是亲兄弟了，但他俩毕竟还有一些区别：
树状数组能有的操作，线段树一定有；线段树有的操作，树状数组不一定有.

树状数组的代码要比线段树短得多，思维也更清晰，在解决一些单点修改的问题时，树状数组是不二之选。

举例：
%%html
<img src="https://pic4.zhimg.com/80/v2-1c3424a82a41428901f94e0f7277a3e3_720w.jpg">

树状数组为了节省空间，删去了不必要的结点，将结点数压缩到与数组长度相同。方案是这样的：数组的每个位置代表其在图中垂直向上追溯可以达到的最高的结点。如上图，图中的圆圈对应的结点就是数组中位置所代表的结点。比如说数组中的位置 4 就代表了结点 [1, 4]，数组中位置 6 代表了结点 [5, 6]，位置 8 代表了结点 [1, 8]

这么安排下来，得到的数组有一个特点：对于处于数组位置 i 的结点，其代表的信息区间为 [i - lowbit(i) + 1, i]。比如说对于位置 6，其二进制表示为 110，它所代表的也就是 [101, 110] 这两个元素的信息整合；对于位置8，其二进制表示为 1000，它所代表的也就是 [0001, 1000] 这八个元素的信息整合。

之所以会有这个特点，是因为对于位置 i，其对应的结点所在的高度就是 lowbit(i) 的位数。第一层结点为全体2^0+2^1k，即所有 lowbit(i)=1 的数字；第二层结点为全体2^1+2^2k，即所有 lowbit(i)=2 的数字；第三层结点为全体2^2+2^3k，即所有 lowbit(i)=4 的数字；以此类推。也就是说，对于位置 i，在这个位置往上垂直追溯，能追溯的层数就是 i 的二进制表示的末尾 0 数量。而结点高度又决定了其子树的大小，于是它所代表的信息区间大小也就一定是2^{i的末尾0的数量} = lowbit(i)。

那么再来看我们如何利用这个简化的结点数组进行查询。先明确一下任务：查询树状数组时，我们是要查询区间 [1, i] 的整合信息。查询方法很简单，反复令 i = i - lowbit(i)，整合路径上的信息，直到 i == 0 为止。为什么可以这么搞？我给 f(i) = i - lowbit(i) 画个图就很显然了：

%%html
<img src="https://pic2.zhimg.com/80/v2-b3f7bccdbbe043154137e123b7e2bfbb_720w.jpg">
图中每一个结点 i 上的红色箭头指向它的 i - lowbit(i) 的结果。可以看到，一个结点 i 的位置减去 lowbit(i) 之后，其实就代表着向左离开它所表示的信息区间。比如说如果 i=7，那么由之前所说的我们知道 i 的信息区间长度为 lowbit(7)=1， 那么我们记录下 [7, 7] 区间的信息，开始往回走。往回走就是减去当前信息区间的长度，到达刚才没有覆盖的区间，也就刚好是减去 lowbit(7)=1得到 i=6；然后 i 的信息区间长度为 lowbit(6)=2， 那么我们用 [5, 6] 区间的信息更新手上的信息，继续往回走，也就是减去 lowbit(6)=2得到 i=4；然后 i=4，区间长度为lowbit(4)=4，再用 [1, 4] 的信息来更新手上的信息，继续往前走，减去 lowbit(4)=4得到 i=0；此时说明我们已经考虑到了整个的查询区间，不需要继续了。

那么更新呢？思路很简单：如果是线段树的话，我们是一直往父结点回溯直至根结点，而树状数组与之相比只是少了一些父结点，我们只要每次设法得到下一个存在的祖先结点即可。而对于结点 i，它下一个存在的祖先结点其实就是 i + lowbit(i)。继续看图：

%%html
<img src="https://pic1.zhimg.com/v2-ed952086cb6763440c1cd864cd6c1407_r.jpg">

为什么呢？很简单，要找到 i 的第一个祖先，我们只需要从 i 一直往右走，找到第一个层数比 i 高的结点即可。由于所有的树结点都是向右整合的，这样找到的结点的信息区间一定覆盖 i 结点的信息区间。而既然 lowbit(i) 决定了位置 i 的结点高度，那么我们要找的其实就是第一个使得 lowbit(j) > lowbit(i)的结点 j。如何得到这样一个 j？那就直接以消去 i 的 lowbit 为目标，直接加上一个能够使得 i 的 lowbit 增大的最小的数字，很显然，这个数字也就是 lowbit(i)。这样一来，i+lowbit(i) 就是我们要找的下一个结点，如此往复向上寻找直到到达根结点即可。


In [ ]:
def query(nums, a):
    result = 0
    while a:
        result += nums[a]
        a -= a&(-a)
    return result

def update(nums, a, value):
    while a< len(nums):
        nums[a] += value
        a += a&(-a)


In [ ]:
# 建树 o(n)
def creat(nums):
    t = [0]*len(nums)
    for i, num in enumerate(nums):
        t[i] = nums[i]
        if i + i&(-i) <= n:
            t[i + i&(-i)] += t[i]

$O(log n)$ 查询第k小/大元素。在此处只讨论第k小，第k大问题可以通过简单计算转化为第  小问题。关于求区间第k小的思想。将所有数字看成一个可重集合，即定义数组a表示值为i的元素在整个序列重出现了$a_i$次。找第k大就是找到最小的x恰好满$\sum_{i=1}^xa_i \ge k$

因此可以想到算法：如果已经找到x满足$\sum_{i=1}^xa_i \le k$，考虑能不能让x继续增加，使其仍然满足这个条件。找到最大的x后，x+1就是所要的值。 在树状数组中，节点是根据2的幂划分的，每次可以扩大2的幂的长度。令sum表示当前的x所代表的前缀和，有如下算法找到最大的x ：

1. 求出depth = floor($log_2n$)
2. 计算t=$\sum_{i=x+1}^{x+2^{depth}}a_i$
3. 如果$sum + t \le k$, 则此时扩展成功， 将$2^{depth}$累加到x上；否则扩展失败，对x不进行操作
4. 将depth减去1， 回到步骤2，直至depth为0

In [51]:
def kth(nums, k):
    cnt, result = 0, 0
    for i in range(log2(len(nums)), ~i, -1):
        result += 1<<i
        if result >= len(nums) or cnt + nums[result] >= k:
            result -= 1<<i
        else:
            cnt += nums[result]
    return result + 1

时间戳优化：

对付多组数据很常见的技巧。如果每次输入新数据时，都暴力清空树状数组，就可能会造成超时。因此使用tag标记，存储当前节点上次使用时间（即最近一次是被第几组数据使用）。每次操作时判断这个位置tag中的时间和当前时间是否相同，就可以判断这个位置应该是 0 还是数组内的值。

# 线段树

线段树可以在$O(logN)$的时间复杂度内实现单点修改、区间修改、区间查询（区间求和，求区间最大值，求区间最小值）等操作。

线段树维护的信息，需要满足可加性，即能以可以接受的速度合并信息和修改信息，包括在使用懒惰标记时，标记也要满足可加性（例如取模就不满足可加性，对4取模然后对3取模，两个操作就不能合并在一起做


## 线段树的基本结构与建树
线段树将每个长度不为1的区间划分成左右两个区间递归求解，把整个线段划分为一个树形结构，通过合并左右两区间信息来求得该区间的信息。这种数据结构可以方便的进行大部分的区间操作。
有个大小为5的数组$a=\{10,11,12,13,14\}$，要将其转化为线段树，有以下做法：设线段树的根节点编号为1，用数组d来保存我们的线段树，$di$用来保存线段树上编号为i的节点的值（这里每个节点所维护的值就是这个节点所表示的区间总和），如图所示：

%%html
<img src="https://oi-wiki.org/ds/images/segt1.png">
图中$d_i1$表示根节点，紫色方框是数组a，红色方框是数组d，红色方框中的括号中的黄色数字表示它所在的那个红色方框表示的线段树节点所表示的区间，如$d_1$所表示的区间就是[1,5], 即  所保存的值是$a_1+a_2+a_3+\cdots+a_5$, $d_1=60$表示的是$a_1+a_2+a_3+\cdots+a_5=60$

观察不难发现，$d_i$的左儿子节点是$d_{2i}$, 右儿子节点是$d_{2i+1}$. 如果$d_{i}$表示区间$[s,t]$, 即$d_i=a_s+\cdots+a_t$, 那么$d_i$的左儿子节点表示的是区间$[s, \frac{s+t}{2}]$, $d_i$的右儿子表示的区间$[\frac{s+t}{2}+1,t]$, $d_i$表示区间长度为$t-s+1$, 当$d_i$表示区间大小为1时, $d_i=a_s=s_t$, 这就是线段树的递归边界。

In [ ]:
def build(s, t, p):
    if s==t:
        d[p] = a[s]
        return
    m = (s+t)//2
    build(s, m, p*2)
    build(m+1, t, p*2+1)
    d[p] = d[p*2] + d[p*2+1]


如果线段树采用堆方式存储(2p是left child, 2p+1是right child), 若有n个页节点，则d数组的范围最大是$2^{ceiling(log n)+1}$

## 线段树的区间查询
区间查询，比如求区间$\[l,r\]$的总和(即$a_l+a_{l+1}+\cdot+a_r)$,求区间最大值/最小值等操作.

%%html
<img src='https://oi-wiki.org/ds/images/segt5.png' height= 500, width=500>

以上面这张图为例，如果要查询区间$[1,5]$的和，那直接获取$d_1$的值即可。
如果要查询的区间为$[3,5]$，此时就不能直接获取区间的值，但是$[3,5]$可以拆成$[3,3]$和$[4,5]$，可以通过合并这两个区间的答案来求得这个区间的答案. 一般地，如果要查询的区间是$[l,r]$，则可以将其拆成最多为$O(log n)$个极大的区间，合并这些区间即可求出$[l,r]$的答案.

In [ ]:
def query(left, right, s, t, p):
    if left<=s and t<=right:
        return d[p]
    m, result = (s+t)//2, 0
    if left <= m:
        result += query(left, right, s, m, p*2)
    if right > m:
        result += query(left, right, m+1, t, p*2+1)
    return result

In [52]:
class SegmentTree:
    """线段树类"""
    def __init__(self, alist, merger_):
        """
        Description: 线段树的构造函数
        Params:
        - alist: 用户传入的一个list（这里我们就不用以前实现的Arr类了，直接用python的list啦，如果想用的话也是一点问题都没有的～）
        - func: merge函数，用于对实现两个数合成一个数的功能（比如二元操作符加法、乘法……等等）
        """
        self._data = alist[:]   # 所以为了不改变传入的数组，需要传其副本
        self._tree = [None] * 4 * len(self._data)       # 注意是4倍的存储空间，初始化元素全是None
        # self._tree = [None for i in range(len(self._data) * 4)]
        self._merger = merger_   # merger函数，比如两个元素求和函数……，用lambda表达式比较方便

        self._buildSegmentTree(0, 0, len(self._data)-1) # 调用self._buildSegmentTree来构建线段树

    def getSize(self):
        """
        Description: 获取有效元素的个数
        Returns:
        有效元素个数
        """
        return len(self._data)

    def get(self, index):
        """
        Description: 根据索引index获取相应元素
        时间复杂度：O(1)
        Params:
        - index: 传入的索引
        Returns:
        index索引处的元素值
        """
        if index < 0 or index >= len(self._data):
            raise Exception('Index is illegal!')
        return self._data[index]

    def query(self, quaryL, quaryR):
        """
        Description: 查找[quaryL, quaryR]这个左闭右闭区间上的值（例如对于求和操作就是求这个区间上所有元素的和）
        时间复杂度：O(logn)
        Params:
        - quaryL: 区间左端点的索引
        - quaryR: 区间右端点的索引
        Returns:
        [quaryL, quaryR]区间上的值
        """
        if quaryL < 0 or quaryR < 0 or quaryL >= self.getSize() or quaryR >= self.getSize() or quaryR < quaryL:  # 索引合法性检查
            raise Exception('The indexes is illegal!')
        return self._query(0, 0, self.getSize()-1, quaryL, quaryR)  # 调用self._quary函数

    def set(self, index, e):
        """
        Description: 将数组中index位置的元素设为e，因此此时需要对线段树的内容要进行更新操作(也就是线段树的更新操作)
        时间复杂度：O(logn)
        Params:
        - index: 数组中的索引
        - e: 索引index上元素的新值e
        """
        if index < 0 or index >= self.getSize():
            raise Exception('The index is illegal!')
        self._data[index] = e # 更新self._data
        self._set(0, 0, len(self._data) - 1, index, e)  # 调用self._set函数
        

    def printSegmentTree(self):
        """对线段树进行打印"""
        print('[', end=' ')
        for i in range(len(self._tree)):
            if i == len(self._tree) - 1:
                print(self._tree[i], end=' ]')
                break
            print(self._tree[i], end=',')


    # private
    def _leftChild(self, index):
        """
        Description: 和最大堆一样，由于线段树是一颗完全二叉树，所以可以通过索引的方式找到其左、右孩子的索引（元素从索引0开始盛放）
        Params:
        - index: 输入的索引
        Returns:
        左孩子的索引值
        """
        return 2 * index + 1		# 一定要记住线段树是一棵满树哦，所以用数组就能表示这棵树了，索引关系也和堆是一样的，只不过不需要求父亲节点的索引了

    def _rightChild(self, index):
        """
        Description: 和最大堆一样，由于线段树是一颗完全二叉树，所以可以通过索引的方式找到其左、右孩子的索引（元素从索引0开始盛放）
        Params:
        - index: 输入的索引
        Returns:
        右孩子的索引值
        """
        return 2 * index + 2

    def _buildSegmentTree(self, treeIndex, left, right):
        """
        Description: 以根节点索引为treeIndex，构造self._data索引在[left, right]上的线段树
        Params:
        - treeIndex: 线段树根节点的索引
        - left: 数据左边的索引
        - right: 数据右边的索引
        """
        if left == right:       # 递归到底的情况，left == right，此时只有一个元素
            self._tree[treeIndex] = self._data[left]  # 相应的，self._tree上索引为treeIndex的位置的值置为self._data[left]就好
            return 

        leftChild_index = self._leftChild(treeIndex)    # 获取左孩子的索引
        rightChild_index = self._rightChild(treeIndex)  # 获取右孩子的索引
        
        mid = left + (right - left) // 2        # 获取left和right的中间值，在python中，可以用(left + right) // 2的方式来获得mid，因为不存在数值越界问题
        self._buildSegmentTree(leftChild_index, left, mid)  # 递归向左孩子为根的左子树构建线段树
        self._buildSegmentTree(rightChild_index, mid + 1, right)  # 递归向右孩子为的右子树构建线段树
        self._tree[treeIndex] = self._merger(self._tree[leftChild_index], self._tree[rightChild_index]) # 在回归的过程中，用self._merger函数对两个子节点的值进行merger操作，从而完成整棵树的建立
        
    def _query(self, treeIndex, left, right, quaryL, quaryR):
        """
        Description: 在根节点索引为treeindex的线段树上查找索引范围为[quaryL, quaryR]上的值，其中left， right值代表该节点所表示的索引范围（左闭右闭）
        Params:
        - treeIndex: 根节点所在的索引
        - left: 根节点所代表的区间的左端的索引值(注意是左闭右闭区间哦)
        - right: 根节点所代表的区间的右端点的索引值
        - quaryL: 待查询区间的左端的索引值（也是左闭右闭区间）
        - quaryR: 待查询区间的右端的索引值
        """
        if left == quaryL and right == quaryR:      # 递归到底的情况，区间都对上了，直接返回当前treeIndex索引处的值就好
            return self._tree[treeIndex]            # 返回当前树上索引为treeIndex的元素值
        
        mid = left + (right - left) // 2            # 获取TreeIndex索引处所代表的范围的中点
        leftChild_index = self._leftChild(treeIndex)    # 获取左孩子的索引
        rightChild_index = self._rightChild(treeIndex)  # 获取右孩子的索引

        if quaryL > mid:        # 此时要查询的区间完全位于当前treeIndex所带表的区间的右侧
            return self._query(rightChild_index, mid + 1, right, quaryL, quaryR)    # 直接去右子树找[quaryL, quaryR]
        elif quaryR <= mid:     # 此时要查询的区间完全位于当前treIndex所代表的区间的左侧
            return self._query(leftChild_index, left, mid, quaryL, quaryR)      # 直接去左子树找[quaryL, quaryR]
        
        # 此时一部分在[left, mid]上，一部分在[mid + 1, right]上
        leftResult = self._query(leftChild_index, left, mid, quaryL, mid)   # 在左子树找区间[quaryL, mid]
        rightResult = self._query(rightChild_index, mid + 1, right, mid + 1, quaryR)    # 在右子树找区间[mid + 1, quaryR]
        return self._merger(leftResult, rightResult)        # 最后在回归的过程中两个子节点进行merger操作并返回,得到[quaryL, quaryR]区间上的值

    def _set(self, treeIndex, left, right, index, e):
        """
        Description: 在以索引treeIndex为根节点的线段树中将索引为index的位置的元素设为e（此时treeIndex索引处所代表的区间范围为：[left, right]
        params:
        - treeIndex: 传入的线段树的根节点索引值
        - left: 根节点所代表的区间的左端的索引值
        - right: 根节点所代表的区间的右端点的索引值
        - index: 输入的索引值
        - e: 新的元素值
        """
        if left == right:  # 递归到底的情况，也就是在树中找到了索引为index的元素
            self._tree[treeIndex] = e  # 直接替换
            return

        mid = left + (right - left) // 2        # 找到索引中间值
        leftChild_index = self._leftChild(treeIndex)    # 左孩子索引值
        rightChild_index = self._rightChild(treeIndex)  # 右孩子索引值

        if index <= mid:    # index处于当前treeIndex所代表的区间的左半区
            self._set(leftChild_index, left, mid, index, e) # 到左子树去找index
        else:       # 否则index处于当前treeIndex所代表的区间的右半区
            self._set(rightChild_index, mid + 1, right, index, e)   # 到右子树去找index
        self._tree[treeIndex] = self._merger(self._tree[leftChild_index], self._tree[rightChild_index]) # 由于对树的最底层元素进行了更新操作，因此需要对树的上层也进行一次更新，所以每次回归的都调用merger操作进行上层的值的更新操作
